In [2]:
import numpy as np
import scipy as sp
import scipy.linalg as la
import numpy.matlib as ml

In [3]:
A = np.array([[1,0,0,0,0,0],
     [0,2,0,0,0,0],
     [0,0,3,0,0,0],
     [0,0,0,4,0,0],
     [0,0,0,0,5,0],
     [0,0,0,0,0,6],
    ])
x = np.array([[0.3804],
              [0.5678],
              [0.0759],
              [0.0540],
              [0.5308],
              [0.7792]
    ])

m = 4
a = 2.5
b = 5.5
a0 = 1
expected =  [ [0.3804], [0.07060463], [-0.00076315], [0.0005567], [-0.00533705], [0.09689174]]

In [4]:
def chebyshev_filter(A, x,m,a,b,a0):
    e = (b - a)/2
    c = (b + a)/2
    sigma = e/(a0 - c)
    sigma_1 = sigma;
    y = (np.dot(A,x) - c*x)*sigma_1/e
    for i in range (1,m):
        sig_new = 1 / (2/sigma_1 - sigma)
        y_new = 2*(np.dot(A,y) - c*y)*sig_new/e - sigma*sig_new*x
        x = y
        y = y_new
        sigma = sig_new;
    return y

chebyshev_filter_out = chebyshev_filter(A, x,m,a,b,a0)
print(all(abs(chebyshev_filter_out - expected) < 1e-5))

True


In [53]:
B = np.array([[1,0,0,0,0,0],[1,0,0,0,0,0],[1,0,0,0,0,0],[1,0,0,0,0,0],[1,0,0,0,0,0],[1,0,0,0,0,0]])
display(x, x.shape)
display(x.T, x.T.shape)
display(B)
display(B.T)
display(B[0,:])
display(np.dot(B,x))
display(np.dot(x.T,B))

array([[0.3804],
       [0.5678],
       [0.0759],
       [0.054 ],
       [0.5308],
       [0.7792]])

(6, 1)

array([[0.3804, 0.5678, 0.0759, 0.054 , 0.5308, 0.7792]])

(1, 6)

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0]])

array([[1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

array([1, 0, 0, 0, 0, 0])

array([[0.3804],
       [0.3804],
       [0.3804],
       [0.3804],
       [0.3804],
       [0.3804]])

array([[2.3881, 0.    , 0.    , 0.    , 0.    , 0.    ]])

In [108]:
def cheby_dav_eigh(A, x_vec, m, kwant, kkeep, dim_max, tol):
    eig_val = np.zeros(A.shape[1])
    kc = 0
    iter_max = 100000

    V = np.zeros(A.shape,dtype=float)
    W = np.zeros(A.shape,dtype=float)
    H = np.zeros(A.shape,dtype=float)
    
    x = x_vec/la.norm(x_vec)
    x_arr = np.array(x)
    V[:,0] = x[:,0]
    w = np.dot(A,x)
    W[:,0] = w[:,0]
    
    mu = np.dot(x.T,w)[0,0]
    H[0,0] = mu

    r = W[:,0] - mu*x
    
    if la.norm(r) <= tol:
        eig_val[0] = mu
        kc = 1
        H = np.array([])
    else:
        kc= 0
    #print('H',H)
    
    #TODO: esimate upper bound of eigenvalues
    upperb = 10
    lowerb = (upperb+mu)/2
    a0 = lowerb
    
    ksub = 0
    iter = 0
    while iter <= iter_max:
        iter+=1
        if iter % 1000 == 0:
            print('ITERATION',iter)
        t = chebyshev_filter(A, x, m, lowerb, upperb, a0)
        
        #orthogonalize t against first k_sub columns of V by householder
        for c in range(ksub):
            v  = V[:,c]
            v = v.reshape(v.size,1)
            t = t - v * np.dot(v.T, t)[0]/np.dot(v.T,v)
        t = t/la.norm(t)
        V[:,ksub+1] = t[:,0]
        ksub = ksub + 1;
        kold = ksub
        W[:,ksub] = A @ V [:, ksub]
        H[0 : ksub - kc+1, ksub - kc] = V[:, kc : ksub+1].T @ W[:, ksub]
        H[ksub - kc, 0 : ksub - kc+1] = H[0 : ksub - kc+1, ksub - kc].T
        
        eigVal,Y = la.eigh(H[0 : ksub - kc+1,0 : ksub - kc+1])
        D = np.diag(eigVal)
        
        if ksub >= dim_max:
            ksub = kc + kkeep
    
        V[:, kc : ksub + 1] = V[:, kc : kold + 1] @ Y[:, 0 : ksub - kc + 1]
        
        W[:, kc : ksub + 1] = W[:, kc : kold + 1] @ Y[:, 0 : ksub - kc + 1]
        
        r = W[:, kc] - mu * V[:, kc]
        
        noswap = 0
        if la.norm(r) <= tol * max(np.diag(D)):
            kc = kc + 1
            print("CONVERGED:",kc)
        
        eig_val[kc] = mu
        vtmp = V[:,kc]
        
        i = kc
        while eig_val[i-1] > mu:
            eig_val[i] = eig_val[i-1]
            V[:,i] = V[:,i-1]
            noswap=1 
            i -= 1
        eig_val[i] = mu
        if noswap == 1:
            V[:,i] = vtmp
        
        
        if (kc >= kwant and noswap == 0):
            return eig_val[0 : kc+1] and V[:, 0:kc+1]

        lowerb = np.median(np.diag(D))
        if a0 > min(np.diag(D)):
            a0 = min(np.diag(D))
            
        x = V[:, kc]
        H = np.zeros(A.shape,dtype=float)
        H[kc : ksub+1, kc : ksub+1] = D[kc : ksub+1, kc : ksub+1]
        
x_vec = x
tol = 1e-12
cheby_dav_eigh(A, x_vec, m, 4, 2, 5, tol)

ITERATION 1000
ITERATION 2000
ITERATION 3000
ITERATION 4000
ITERATION 5000
ITERATION 6000
ITERATION 7000
ITERATION 8000
ITERATION 9000
ITERATION 10000
ITERATION 11000
ITERATION 12000
ITERATION 13000
ITERATION 14000
ITERATION 15000
ITERATION 16000
ITERATION 17000
ITERATION 18000
ITERATION 19000
ITERATION 20000
ITERATION 21000
ITERATION 22000
ITERATION 23000
ITERATION 24000
ITERATION 25000
ITERATION 26000
ITERATION 27000
ITERATION 28000
ITERATION 29000
ITERATION 30000
ITERATION 31000
ITERATION 32000
ITERATION 33000
ITERATION 34000
ITERATION 35000
ITERATION 36000
ITERATION 37000
ITERATION 38000
ITERATION 39000
ITERATION 40000
ITERATION 41000
ITERATION 42000
ITERATION 43000
ITERATION 44000
ITERATION 45000
ITERATION 46000
ITERATION 47000
ITERATION 48000
ITERATION 49000
ITERATION 50000
ITERATION 51000
ITERATION 52000
ITERATION 53000
ITERATION 54000
ITERATION 55000
ITERATION 56000
ITERATION 57000
ITERATION 58000
ITERATION 59000
ITERATION 60000
ITERATION 61000
ITERATION 62000
ITERATION 63000
I

In [26]:
B=A
padB = [[0],[0],[0],[0],[0],[0]]
np.pad(B, ((0,0),(0,1)), 'constant', constant_values=0)

array([[1, 0, 0, 0, 0, 0, 0],
       [0, 2, 0, 0, 0, 0, 0],
       [0, 0, 3, 0, 0, 0, 0],
       [0, 0, 0, 4, 0, 0, 0],
       [0, 0, 0, 0, 5, 0, 0],
       [0, 0, 0, 0, 0, 6, 0]])

In [55]:

W=np.random.rand(6,2)
print(W)
print(A[:,2:4])
print(A[:,2:4].T @ W)

[[0.80231364 0.51122494]
 [0.55727141 0.54403408]
 [0.98821887 0.34084358]
 [0.2798732  0.66829121]
 [0.93634563 0.19028284]
 [0.96706293 0.30129424]]
[[0 1]
 [0 1]
 [3 0]
 [0 4]
 [0 0]
 [0 0]]
[[2.96465661 1.02253073]
 [2.47907785 3.72842387]]


In [ ]:
A[:,]

In [54]:
B=A
print(B)
B[0:2,3] = np.array([1, 1])
B

[[1 0 0 0 0 0]
 [0 2 0 0 0 0]
 [0 0 3 0 0 0]
 [0 0 0 4 0 0]
 [0 0 0 0 5 0]
 [0 0 0 0 0 6]]


array([[1, 0, 0, 1, 0, 0],
       [0, 2, 0, 1, 0, 0],
       [0, 0, 3, 0, 0, 0],
       [0, 0, 0, 4, 0, 0],
       [0, 0, 0, 0, 5, 0],
       [0, 0, 0, 0, 0, 6]])

In [ ]:
t = x
for c in range(A.shape[1]):
    v = A[:,c].T
    v = v.reshape(v.size,1)
    t = t - v * np.dot(v.T, t)[0][0]/np.dot(v.T,v)
    print('dot product',np.dot(v.T,t)[0][0])

In [ ]:
linalg.norm(x)

In [ ]:
s = 0
for a in x:
    s += a[0]*a[0]
np.sqrt(s)

In [ ]:
W[:,0]

In [ ]:
r